In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


The core idea is self attention
Transfomer create s stacks of self attention layers and explained the scaled dot product attention  and multi-head attention

The Transfomer layers can substitute rnn or cnn


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import time
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# Get Portugese-English tranlation dataset 

examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXHJFBL/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXHJFBL/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXHJFBL/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [5]:
metadata

tfds.core.DatasetInfo(
    name='ted_hrlr_translate',
    version=1.0.0,
    description='Data sets derived from TED talk transcripts for comparing similar language pairs
where one is high resource and the other is low resource.
',
    homepage='https://github.com/neulab/word-embeddings-for-nmt',
    features=Translation({
        'en': Text(shape=(), dtype=tf.string),
        'pt': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=54781,
    splits={
        'test': 1803,
        'train': 51785,
        'validation': 1193,
    },
    supervised_keys=('pt', 'en'),
    citation="""@inproceedings{Ye2018WordEmbeddings,
      author  = {Ye, Qi and Devendra, Sachan and Matthieu, Felix and Sarguna, Padmanabhan and Graham, Neubig},
      title   = {When and Why are pre-trained word embeddings useful for Neural Machine Translation},
      booktitle = {HLT-NAACL},
      year    = {2018},
      }""",
    redistribution_info=,
)

In [0]:
train_examples, val_examples = examples['train'], examples['validation']

In [9]:
# Create custome subword tokenizer from dataset

train_examples

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>

In [0]:
for n, pt, en in enumerate(train_examples):
  print(pt)
  print('--')
  print(en)

  if n == 0:
    break

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus((en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus((pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

In [12]:
sample_string = 'Transfomer is aweosme'

tokenized_string = tokenizer_en.encode(sample_string)
print(tokenized_string)

decoded_string = tokenizer_en.decode(tokenized_string)
print(decoded_string)

[7915, 1248, 7946, 1391, 2762, 7863, 13, 2497, 2629, 1028, 7932]
Transfomer is aweosme


In [17]:
for ts in tokenized_string:
  print(ts,'\t',tokenizer_en.decode([ts]))

7915 	 T
1248 	 ran
7946 	 s
1391 	 fo
2762 	 mer
7863 	  
13 	 is 
2497 	 aw
2629 	 eo
1028 	 sm
7932 	 e


In [18]:
tokenizer_pt.vocab_size

8214

In [0]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64


# Add start and end token 
def encode(lang1, lang2):
  lang1 = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(lang1.numpy()) + [tokenizer_pt.vocab_size + 1]  # the tokenizer_pt.vocab_size = sos 
  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(lang2.numpy()) + [tokenizer_en.vocab_size + 1] # the tokenizer_pt.vocab_size + 1 = eos

  return lang1, lang2

In [0]:
def tf_encode(pt, en):
  result_pt, result_en = tf.py_function(encode, [pt, en], [tf.int64, tf.int64])
  result_pt.set_shape([None])
  result_en.set_shape([None])
  return result_pt, result_en

In [0]:
MAX_LENGTH = 40

In [0]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
  '''
  filter the max length is less than MAX_LENGTH
  '''
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [0]:
train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, ([None], [None]))  # requires to set each elements
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE, ([None], [None]))

In [74]:
train_dataset

<DatasetV1Adapter shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

In [77]:
for n, (pt, en) in enumerate(train_dataset):
  print(pt)
  print('--')
  print(en)

  if n == 0:
    break

tf.Tensor(
[[8214 1231  258 ...    0    0    0]
 [8214   27 2613 ...    0    0    0]
 [8214  125  410 ...    0    0    0]
 ...
 [8214   24   98 ...    0    0    0]
 [8214  374    1 ...    0    0    0]
 [8214    8   10 ...    0    0    0]], shape=(64, 35), dtype=int64)
--
tf.Tensor(
[[8087   14   88 ...    0    0    0]
 [8087   12   20 ...    0    0    0]
 [8087  145   14 ...    0    0    0]
 ...
 [8087   11   43 ...    0    0    0]
 [8087 3303   12 ... 8088    0    0]
 [8087   17   13 ...    0    0    0]], shape=(64, 35), dtype=int64)


## Requires to do the positional Encoding